In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from google.colab import files
uploaded = files.upload()

Saving cleaned_tracks_genre.csv to cleaned_tracks_genre.csv


In [0]:
track_table = pd.read_csv("cleaned_tracks_genre.csv")
track_table.dropna(inplace=True)

In [0]:
track_table.groupby("master_popular_genre").track_id.count()

master_popular_genre
alternative     4545
country         7238
hip-hop         8144
house           5719
indie           5741
pop            17986
r&b             3678
rock            5298
Name: track_id, dtype: int64

In [0]:
track_table.track_year = track_table.track_year.astype(int)

In [0]:
#track_table.drop("master_popular_genre", axis=1, inplace=True)
#track_table = track_table[(track_table['track_spotify_genre'] != 'trap') & (track_table['track_spotify_genre'] != 'rap')]

In [0]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data_numerics = pd.concat([track_table.select_dtypes(include=[np.number]),track_table['master_popular_genre']],axis=1, sort=False)

In [0]:
data_numerics.head()

,track_year,track_pop,duration_ms,time_signature,key,loudness,energy,speechiness,acousticness,mode,tempo,valence,danceability,instrumentalness,liveness,master_popular_genre
0,2011,77,218013.0,4.0,3.0,-5.835,0.936,0.0439,0.00847,1.0,112.960,0.371,0.560,0.000000,0.1610,rock
1,2011,82,239600.0,4.0,5.0,-5.849,0.710,0.0292,0.14500,0.0,127.975,0.965,0.733,0.115000,0.0956,rock
2,2011,77,235320.0,4.0,2.0,-6.903,0.427,0.1060,0.05310,1.0,93.004,0.216,0.831,0.000000,0.3070,hip-hop
3,2011,77,285040.0,4.0,9.0,-8.251,0.330,0.0285,0.89200,1.0,135.109,0.285,0.559,0.000000,0.0975,pop
4,2011,74,244973.0,4.0,0.0,-7.036,0.495,0.0370,0.59100,1.0,129.062,0.720,0.864,0.000133,0.0992,pop


In [0]:
#track_table_numerics = pd.concat([data_numerics.drop(['master_popular_genre'],axis=1), pd.get_dummies(data_numerics['master_popular_genre'])], axis=1, sort= False)

track_table_dummies = pd.get_dummies(data_numerics,columns=['key','time_signature',],drop_first=True)
track_table_dummies = track_table_dummies.drop('loudness', axis=1)


In [0]:
track_table_dummies.head()

,track_year,track_pop,duration_ms,energy,speechiness,acousticness,mode,tempo,valence,danceability,instrumentalness,liveness,master_popular_genre,key_1.0,key_2.0,key_3.0,key_4.0,key_5.0,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,time_signature_1.0,time_signature_3.0,time_signature_4.0,time_signature_5.0
0,2011,77,218013.0,0.936,0.0439,0.00847,1.0,112.960,0.371,0.560,0.000000,0.1610,rock,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,2011,82,239600.0,0.710,0.0292,0.14500,0.0,127.975,0.965,0.733,0.115000,0.0956,rock,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,2011,77,235320.0,0.427,0.1060,0.05310,1.0,93.004,0.216,0.831,0.000000,0.3070,hip-hop,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,2011,77,285040.0,0.330,0.0285,0.89200,1.0,135.109,0.285,0.559,0.000000,0.0975,pop,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,2011,74,244973.0,0.495,0.0370,0.59100,1.0,129.062,0.720,0.864,0.000133,0.0992,pop,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
genre = track_table_dummies['master_popular_genre']
track_table_dummies.drop('master_popular_genre', axis=1,inplace = True)
track_table_dummies.insert(0, 'master_popular_genre', genre)

In [0]:
track_table_dummies.head()

,master_popular_genre,track_year,track_pop,duration_ms,energy,speechiness,acousticness,mode,tempo,valence,danceability,instrumentalness,liveness,key_1.0,key_2.0,key_3.0,key_4.0,key_5.0,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,time_signature_1.0,time_signature_3.0,time_signature_4.0,time_signature_5.0
0,rock,2011,77,218013.0,0.936,0.0439,0.00847,1.0,112.960,0.371,0.560,0.000000,0.1610,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,rock,2011,82,239600.0,0.710,0.0292,0.14500,0.0,127.975,0.965,0.733,0.115000,0.0956,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,hip-hop,2011,77,235320.0,0.427,0.1060,0.05310,1.0,93.004,0.216,0.831,0.000000,0.3070,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,pop,2011,77,285040.0,0.330,0.0285,0.89200,1.0,135.109,0.285,0.559,0.000000,0.0975,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,pop,2011,74,244973.0,0.495,0.0370,0.59100,1.0,129.062,0.720,0.864,0.000133,0.0992,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
X = track_table_dummies.iloc[:,1:]
y = track_table_dummies.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
y_train.unique()

array(['pop', 'country', 'indie', 'hip-hop', 'rock', 'house',
       'alternative', 'r&b'], dtype=object)

## SVC

In [0]:
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
svm_param_grid = {
    'degree': [2,3,4,5], #[3],
    'kernel': ['rbf'] #['rbf','sigmoid']
}

svm_grid = GridSearchCV( SVC(), svm_param_grid, cv= 3).fit(X_train, y_train)

# summarize the results of the grid search
print("The best score is {}".format(svm_grid.best_score_))
print("The best hyper parameter setting is {}".format(svm_grid.best_params_))

The best score is 0.3941434719951388
The best hyper parameter setting is {'degree': 2, 'kernel': 'rbf'}


In [0]:
print("SVM:")
print("Test accuracy:",accuracy_score(y_test, svm_grid.predict(X_test)))

SVM:
Test accuracy: 0.39719502513892563


In [42]:
gb_param_grid = {
       #'n_estimators': range(80, 150, 20),
       'n_estimators': [140],
       #'max_depth':range(3,8),
       'max_depth':[6],
       #'learning_rate' : [0.1,0.3],
       'learning_rate' : [0.1],
       #'min_samples_split': [0.05, 0.1]
       'min_samples_split': [0.05]

}
grid = GridSearchCV(GradientBoostingClassifier(), gb_param_grid, cv=3).fit(X_train, y_train)


# summarize the results of the grid search
print("The best score is {}".format(grid.best_score_))
print("The best hyper parameter setting is {}".format(grid.best_params_))
print("Model Accuracy:", accuracy_score(y_test,grid.predict(X_test)))

The best score is 0.5263346379147161
The best hyper parameter setting is {'learning_rate': 0.1, 'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 140}
Model Accuracy: 0.5276277523888658


In [43]:
from sklearn.metrics import confusion_matrix
genre_order = ['alternative', 'country', 'hip-hop', 'house', 'indie', 'pop', 'r&b', 'rock']

conf_matrix = confusion_matrix(y_test, grid.predict(X_test), labels=genre_order)
print(conf_matrix)
print("Accuracy predicting Alternative:", conf_matrix[0,0]/(sum(conf_matrix[:,0])))
print("Accuracy predicting Country:", conf_matrix[1,1]/(sum(conf_matrix[:,1])))
print("Accuracy predicting Hip-Hop:", conf_matrix[2,2]/(sum(conf_matrix[:,2])))
print("Accuracy predicting House:", conf_matrix[3,3]/(sum(conf_matrix[:,3])))
print("Accuracy predicting Indie:", conf_matrix[4,4]/(sum(conf_matrix[:,4])))
print("Accuracy predicting Pop:", conf_matrix[5,5]/(sum(conf_matrix[:,5])))
print("Accuracy predicting R&B:", conf_matrix[6,6]/(sum(conf_matrix[:,6])))
print("Accuracy predicting Rock:", conf_matrix[7,7]/(sum(conf_matrix[:,7])))


[[ 378  145   48   87   97  471   96  166]
 [  24 1502   55   32   65  600   25   84]
 [   9   13 1732   47   38  816   44   18]
 [  29   79   40 1256   57  376   26   35]
 [  54  134   75  120  463  716  117  210]
 [ 101  272  641  248  270 4004  184  199]
 [  20   77   96   50  107  496  396   14]
 [ 144  174   31   80  144  685   15  429]]
Accuracy predicting Alternative: 0.4980237154150198
Accuracy predicting Country: 0.6268781302170284
Accuracy predicting Hip-Hop: 0.637233259749816
Accuracy predicting House: 0.6541666666666667
Accuracy predicting Indie: 0.37308622078968573
Accuracy predicting Pop: 0.49044585987261147
Accuracy predicting R&B: 0.43853820598006643
Accuracy predicting Rock: 0.37142857142857144


In [0]:
# Add prediction of 2nd highest probability genre
two_preds = []
for prediction in grid.predict_proba(X_test):
  prediction = list(prediction)
  two_maxes = []
  two_maxes.append(prediction.index(max(prediction)))
  prediction[prediction.index(max(prediction))] = 0
  two_maxes.append(prediction.index(max(prediction)))
  two_preds.append(two_maxes)

In [0]:
# Convert probability to genre name
for pred in two_preds:
  pred[0] = genre_order[pred[0]]
  pred[1] = genre_order[pred[1]]

In [0]:
# Create list to evaluate accuracy
in_list = []
for i in range(0,len(two_preds)):
  in_list.append(int(list(y_test)[i] in two_preds[i]))

In [51]:
print("Accuracy:" , sum(in_list) / len(in_list))

Accuracy: 0.7333818030743664


In [0]:
pred_correctly = []
true_genre = []
predicted_genre = []

for i in range(0,len(y_test)):
  pred_correctly.append(in_list[i])
  true_genre.append(list(y_test)[i])
  predicted_genre.append(two_preds[i])

prediction_frame = pd.DataFrame({"Predicted_Correctly":pred_correctly,
                                 "True Genre": true_genre,
                                 "Predicted Genres": predicted_genre,
                                 "Track Name": track_table.track_name[y_test.index],
                                 "Artist Name": track_table.art_name[y_test.index]})

In [71]:
display(prediction_frame[prediction_frame.Predicted_Correctly == 0])

,Predicted_Correctly,True Genre,Predicted Genres,Track Name,Artist Name
61641,0,country,"[alternative, rock]",Cry Pretty,Carrie Underwood
55514,0,r&b,"[pop, hip-hop]",Lord Is Coming (feat. YBN Cordae),H.E.R.
68885,0,house,"[hip-hop, pop]",Wait A Minute,Party Favor
9185,0,rock,"[pop, alternative]",A Twist in My Story,Secondhand Serenade
4388,0,hip-hop,"[rock, alternative]",She Will,Lil Wayne
...,...,...,...,...,...
72817,0,pop,"[rock, country]",Sanitarios,Caramelos De Cianuro
12621,0,indie,"[rock, pop]",Ode to the Bouncer,Studio Killers
69929,0,house,"[hip-hop, r&b]",wilhelm,deadmau5
57762,0,pop,"[house, alternative]",Twist,Knuckle Puck
